In [1]:
import pandas as pd

In [3]:
channel1 = 'Scikitlearn'

In [4]:
df_profile = pd.read_csv(channel1+'/'+channel1+'_profile.csv')

In [5]:
df_profile.head()

,user_Id,cluster_Id,answer_date,answers
0,54d4a1d6db8155e6700f853b,12,2015-02-06 11:14:00,I'm with family this week and so not super act...
1,54d4a1d6db8155e6700f853b,61,2015-02-24 22:46:00,cool\nok\n
2,54d4a1d6db8155e6700f853b,69,2015-02-25 18:59:00,yeah that sounds like a good idea\n
3,54d4a1d6db8155e6700f853b,72,2015-02-25 19:14:00,you mean == 1\n
4,54d4a1d6db8155e6700f853b,75,2015-02-25 19:19:00,that is also weird ;)\n


In [39]:
df_issue = pd.read_json(channel1+'/'+channel1+'_issue.json')

In [40]:
df_issue.head()

,Id,asker,content,lines,time
0,1,541a528b163965c9bc2053de,Hi all!\n,1,2014-09-18 03:36
1,2,544906e2db8155e6700cdd16,I don't know if this is the right place but I ...,2,2014-10-23 19:59
2,3,544906e2db8155e6700cdd16,My first approach was to convert each feature ...,2,2014-10-23 20:01
3,4,53a5cf04a9176b500d1ced1a,This message is without content. I wonder if a...,1,2015-01-23 15:38
4,5,541a528b163965c9bc2053de,"@mac2bua, I just saw your message. It's probab...",1,2015-01-23 16:49


## Tags Matching

In [32]:
import nltk
from nltk.corpus import stopwords

In [41]:
def tags_match(df,df_tags):
    profile_tags = []
    for answer in df:
        tags_matched = []
    
        if len(str(answer)) == 0:
            profile_tags.append(tags_matched)
            continue
    
        for tag in df_tags:
            if len(str(tag)) <= 1 or str(tag) in stopwords.words('english'):
                continue
            
            if str(tag) in str(answer):
                tags_matched.append(tag)
    
        profile_tags.append(tags_matched)
        print(tags_matched)
    
    return profile_tags

In [6]:
df_tags = pd.read_csv('tags.csv')

In [7]:
len(df_tags)

63581

In [9]:
df_tags.head()

,Id,TagName,Count,ExcerptPostId,WikiPostId,IsModeratorOnly,IsRequired
0,1,.net,322128,3624959.0,3607476.0,NaN,NaN
1,2,html,1141471,3673183.0,3673182.0,NaN,NaN
2,3,javascript,2413015,3624960.0,3607052.0,NaN,NaN
3,4,css,768321,3644670.0,3644669.0,NaN,NaN
4,5,php,1443156,3624936.0,3607050.0,NaN,NaN


In [33]:
profile_tags = []
for answer in df_profile["answers"]:
    tags_matched = []
    
    if len(str(answer)) == 0:
        profile_tags.append(tags_matched)
        continue
    
    for tag in df_tags["TagName"]:
        if len(str(tag)) <= 1 or str(tag) in stopwords.words('english'):
            continue
            
        if str(tag) in str(answer):
            tags_matched.append(tag)
    
    profile_tags.append(tags_matched)
    print(tags_matched)

['git', 'ant', 'com', 'nt', 'import', 'https', 'port', 'ear', 'il', 'conflict', 'release', 'pull', 'http', 'github', 'ca', 'ase', 'el', 'object', 'war', 'thin', 'ps', 'super', 'act', 'su', 'rd', 'rel', 'rds', 'rt', 'next', 'hat', 'xt', 'tps', 'imp', 'ex', 'rr', 'po', 'ray', 'sup', 'scikit-learn', 'arr', 'co', 'om', 'sc', 'ch', 'dt', 'mpo', 'mi', 'dtype', 'hub', 'eas', 'ta']
['co']
['ide', 'go', 'hat', 'ou', 'od', 'goo', 'de']
['mean', 'ou', 'yo']
['ls', 'rd', 'hat', 'lso', 'ei', 'als']
['ide', 'ant', 'nt', 'vi', 'io', 'orm', 'relate', 'implementation', 'transform', 'edi', 'rm', 'el', 'ati', 'em', 'thin', 'vin', 'pre', 'mean', 'args', 'ld', 'rel', 'partial', 'tr', 'rt', 'hat', 'predict', 'rgs', 'par', 'mov', 'ready', 'ovi', 'mo', 'ou', 'imp', 'ed', 'rr', 'maybe', 'lr', 'nsform', 'red', 'co', 'bro', 'ch', 'sf', 'eme', 'kin', 'mi', 'ent', 'know', 'eas', 'de', 'ta']
['git', 'com', 'nt', 'vi', 'io', 'int', 'https', 'ear', 'erp', 'behavior', 'pull', 'http', 'github', 'ca', 'avi', 'sh', 'ps',

['hey']
['time', 'il', 'su', 'go', 'ime', 'ou', 'wil', 'ill']
[]
['il', 'go', 'ou', 'yo', 'aro', 'ill']
['gr']
['il', 'pre', 'su', 'tty', 'hat', 'ss', 'wil', 'ill']
['ip', 'wiki', 'wikipedia', 'http', 'edi', 'dia', 'ed', 'eas']
['nt', 'rest', 'int', 'hat', 'sti', 'st']
['nt', 'io', 'nio', 'get', 'thin', 'dn', 'idn', 'att', 'mu', 'muc', 'ch', 'ent']
['go', 'ou', 'od', 'goo']
['port', 'il', 'el', 'em', 'ld', 'su', 'rte', 'rt', 'hat', 'sti', 'ou', 'ed', 'po', 'sup', 'st', 'co', 'yo', 'jus', 'ill']
[]
['free', 'el', 'fee', 'eel']
['git', 'com', 'nt', 'awk', 'containers', 'struct', 'pi', 'lex', 'skin', 'ping', 'ems', 'war', 'matrix', 'instance', 'em', 'cin', 'csr', 'ps', 'mean', 'rd', 'eps', 'sta', 'ppi', 'step', 'sli', 'ri', 'tr', 'cons', 'hat', 'par', 'contain', 'ex', 'construct', 'isinstance', 'stan', 'mat', 'st', 'ber', 'rs', 'co', 'om', 'plex', 'sp', 'trix', 'ei', 'kin', 'ompl', 'isin', 'ta']
['com', 'lex', 'cin', 'sli', 'ri', 'less', 'ess', 'mo', 'ex', 'rr', 'ss', 'co', 'om', 'plex', 

['git', 'com', 'nt', 'ip', 'io', 'int', 'https', 'ear', 'wiki', 'struct', 'message', 'http', 'github', 'paste', 'sa', 'ps', 'ri', 'instructions', 'tr', 'sage', 'ssa', 'ess', 'tps', 'ed', 'scikit-learn', 'st', 'sag', 'ss', 'co', 'om', 'sc', 'ag', 'instr', 'hub', '20']
['com', 'nt', 'io', 'int', 'sprint', 'el', 'thin', 'abi', 'mean', 'let', 'ri', 'tr', 'hat', 'uti', 'som', 'ou', 'ed', 'lcom', 'st', 'co', 'om', 'sp', 'ag', 'yo', 'nes', 'hey', 'bt', 'know', 'eas', 'ta']
['il', 'push', 'ca', 'el', 'em', 'sh', 'ri', 'tr', 'mo', 'ou', 'ed', 'ogr', 'ag', 'yo', 'ch', 'rise', 'gr', 'ill']
['il', 'build', 'builder', 'ld', 'ri', 'tr', 'der', 'conda', 'co', 'de']
[]
[]
['python', 'unicode', 'vi', 'https', 'orm', 'ear', 'nic', 'il', 'branch', 'http', 'rm', 'build', 'avi', 'ps', 'ld', 'lds', 'tr', 'hat', 'ico', 'norm', 'ode', 'cod', 'tps', 'ou', 'travis-ci', 'scikit-learn', 'st', 'ranch', 'co', 'ff', 'sc', 'ch', 'od', 'yt', 'ravi', 'uff', 'tuf', 'stuff', 'de']
['go', 'ou', 'co', 'od', 'goo']
['find',

['git', 'ant', 'com', 'nt', 'vi', 'https', 'ear', 'ini', 'nic', 'iso', 'branch', 'pi', 'compare', 'get', 'http', 'commit', 'github', 'el', 'init', 'ssi', 'avi', 'rect', 'ps', 'pic', 'act', 'su', 'infinite', 'dir', 'let', 'ctl', 'pick', 'ri', 'tr', 'hat', 'par', 'du', 'inf', 'cherry-pick', 'tps', 'rom', 'ou', 'comm', 'ed', 'rr', 'po', 'scikit-learn', 'tonic', 'red', 'st', 'nite', 'ss', 'ranch', 'rs', 'co', 'om', 'sc', 'aster', 'yo', 'ch', 'pos', 'mi', 'ton', 'ravi', 'soto', 'hub']
[]
['com', 'https', 'ear', 'http', 'project', 'ps', 'proj', 'tps', '.app', '.a', 'scikit-learn', 'appv', 'co', 'om', 'sc', 'yo', 'appveyor', 'pv']
['ant', 'nt', 'io', 'rest', 'int', 'forum', 'il', 'ls', 'get', 'wise', 'ssi', 'sh', 'wan', 'ld', 'quest', 'lso', 'sti', 'uss', 'ape', 'ou', 'st', 'ss', 'sc', 'sp', 'yo', 'que', 'dis', 'rum', 'ill']
['com', 'nt', 'io', 'parameters', 'ram', 'ear', 'implementation', 'il', 'erp', 'reference', 'param', 'utc', 'pac', 'ati', 'ref', 'em', 'pre', 'act', 'su', 'refer', 'ri', 

['linux', 'il', 'tr', 'hat', 'box', 'ox', 'ton', 'ill']
['nt', 'excel', 'el', 'cell', 'ex', 'ent', 'cel']
['np']
[]
[]
['ide', 'io', 'text', 'equation', 'get', 'el', 'uat', 'ati', 'thin', 'min', 'ld', 'tex', 'xt', 'equations', 'ou', 'ex', 'mu', 'st', 'raft', 'co', 'ag', 'yo', 'jus', 'kin', 'mi', 'kind', 'eas', 'de']
['ch']
['branch', 'mean', 'ode', 'cod', 'st', 'ranch', 'co', 'aster', 'ch', 'od', 'de']
['io', 'ria', 'var', 'cal', 'ca', 'col', 'ld', 'go', 'ri', 'cat', 'hat', 'option', 'umn', 'ou', 'take', 'co', 'sp', 'ch', 'iab', 'categorical', 'ta']
['ant', 'com', 'nt', 'wiki', 'cas', 'sam', 'edi', 'move', 'edit', 'ca', 'add', 'ase', 'case', 'dd', 'em', 'sh', 'sa', 'wan', 'ld', 'bold', 'ace', 'rt', 'par', 'icu', 'mov', 'mo', 'ou', 'comm', 'take', 'ed', 'ais', 'cula', 'st', 'co', 'om', 'ag', 'yo', 'jus', 'isin', 'ent', 'eas', 'ta']
['rs']
[]
['time', 'ase', 'find', 'ime', 'ou', 'take', 'yo', 'eas', 'ta']
[]
[]
[]
[]
[]
['ca', 'min', 'minute', 'mi', 'minutes']
[]
['ip', 'io', 'des', 'ref

['nt', 'int', 'port', 'types', 'integer', 'cas', 'casting', 'ca', 'em', 'thin', 'su', 'lts', 'rt', 'sti', 'ode', 'cod', 'som', 'po', 'sup', 'st', 'co', 'om', 'ert', 'od', 'de']
['post', 'ca', 'xa', 'tr', 'ampl', 'ou', 'ex', 'ed', 'po', 'nm', 'st', 'nmf', 'pos']
['nt', 'text', 'su', 'tex', 'rank', 'xt', 'ou', 'ex', 'ed', 'ch', 'que', 'rke', 'iq', 'ent', 'know']
['com', 'il', 'row', 'commit', 'ca', 'line', 'su', 'go', 'far', 'tr', 'hat', 'sti', 'uses', 'ou', 'comm', 'ed', 'rr', 'na', 'arr', 'st', 'co', 'ff', 'om', 'ose', 'hy', 'rro', 'yo', 'las', 'mi', 'uff', 'tuf', 'stuff', 'sts', 'ill', 'know']
[]
['ant', 'nt', 'import', 'port', 'spl', 'trac', 'pi', 'extract', 'pic', 'act', 'pick', 'tr', 'rt', 'xt', 'disp', 'mo', 'imp', 'ex', 'extra', 'po', 'sentence', 'st', 'sp', 'display', 'sente', 'dis', 'mpo', 'ent', 'ta']
['el', 'ri', 'mu']
['il', 'sse', 'env', 'sed', 'build', 'rect', 'ld', 'su', 'dir', 'sta', 'ctl', 'ed', 'na', 'anaconda', 'st', 'conda', 'ss', 'co', 'tal', 'pyenv', 'lli', 'bypass

['nt', 'methods', 'dep', 'diff', 'ndepend', 'el', 'em', 'thin', 'sta', 'pen', 'ri', 'tr', 'uti', 'ou', 'imp', 'ed', 'der', 'stan', 'ods', 'st', 'eve', 'rs', 'co', 'ff', 'yo', 'ch', 'rstan', 'od', 'eme', 'yt', 'ent', 'de', 'ta']
['html', 'modi', 'il', 'file', 'ml', 'rc', 'hat', 'rom', 'mo', 'ou', 'na', 'tm', 'st', 'rs', 'om', 'yo', 'ats', 'mod', 'tml', 'nam', 'od', 'rce']
['nt', 'int', 'ear', 'text', 'model', 'interface', 'rules', 'mode', 'el', 'em', 'thin', 'require', 'sh', 'sa', 'rule', 'word', 'ld', 'su', 'rd', 'tex', 'ace', 'rt', 'hat', 'del', 'xt', 'ode', 'ready', 'mo', 'ou', 'requires', 'imp', 'ex', 'ning', 'ed', 'lr', 'mach', 'lov', 'gensim', 'ord', 'scikit-learn', 'tun', 'st', 'eve', 'ss', 'word2vec', 'sc', 'cl', 'sp', 'ch', 'ats', 'face', 'mod', 'jus', 'ncl', 'lli', 'od', 'eme', 'req', 'glove', 'vec', 'ent', 'de']
['tr']
['put', 'ice', 'sli', 'tr', 'slice', 'hat', 'alu', 'ou', 'output', 'tput', 'tpu', 'val']
['select', 'il', 'probability', 'el', 'agi', 'col', 'abi', 'hat', 'umn

['nt', 'documentation', 'io', 'il', 'trac', 'patch', 'pi', 'document', 'ls', 'cal', 'sample', 'sam', 'image', 'doc', 'extract', 'ca', 'pixel', 'el', 'ati', 'pix', 'sh', 'sa', 'pixels', 'ld', 'act', 'su', 'scale', 'sca', 'tr', 'dimension', 'ime', 'xt', 'mage', 'rom', 'ampl', 'ou', 'take', 'ex', 'rr', 'extra', 'ray', 'arr', 'st', 'ber', 'om', 'sc', 'sp', 'ag', 'ch', 'jus', 'ent', 'wil', 'gr', 'ill', 'ta']
['il', 'channel', 'el', 'xa', 'ampl', 'ex', 'ed', 'gree', 'red', 'st', 'ag', 'ch', 'jus', 'nne', 'wil', 'gr', 'ill']
['html', 'ide', 'string', 'ant', 'nt', 'vi', 'io', 'https', 'ear', 'height', 'trac', 'patch', 'action', 'synth', 'guid', 'module', 'docstring', 'doc', 'http', 'extract', 'ml', 'ca', 'angle', 'shape', 'xa', 'ocs', 'sha', 'sh', 'wan', 'rect', 'cstring', 'vin', 'ps', 'mean', 'act', 'extraction', 'sta', 'width', 'ri', 'tr', 'wid', 'hat', 'du', 'xt', 'uid', 'tps', 'ring', 'ampl', 'mo', 'ape', 'ou', 'ata', 'ex', 'rr', 'der', 'extra', 'synthetic', 'scikit-learn', 'tm', 'stan', '

['com', 'nt', 'io', 'ear', 'ini', 'il', 'trac', 'position', 'model', 'words', 'cut', 'emma', 'cab', 'extract', 'mode', 'ca', 'unc', 'el', 'ice', 'vocabulary', 'em', 'osi', 'filter', 'sa', 'word', 'composition', 'ld', 'act', 'su', 'rd', 'let', 'go', 'days', 'pen', 'ri', 'tr', 'decomposition', 'rds', 'cat', 'hat', 'neat', 'del', 'par', 'xt', 'ode', 'uses', 'mo', 'ets', 'ou', 'comm', 'ecom', 'ex', 'ning', 'ed', 'der', 'extra', 'po', 'eco', 'tire', 'runc', 'ord', 'lte', 'mat', 'red', 'st', 'rs', 'co', 'om', 'sp', 'ch', 'mod', 'jus', 'deco', 'mpo', 'od', 'pos', 'voc', 'filtered', 'emm', 'ent', 'training', 'de', '20']
['il', 'rd', 'ou', 'ch', 'wil', 'ill']
['hat']
[]
['il', 'el', 'su', 'rel', 'hat', 'ou', 'yo', 'wil', 'ill']
['git', 'com', 'https', 'http', 'github', 'em', 'ps', 'su', 'hat', 'pan', 'tps', 'ed', 'pandas', 'mer', 'red', 'ss', 'co', 'om', 'eme', 'hub', 'de']
['il', 'pi', 'ls', 'ca', 'col', 'pic', 'tor', 'cc', 'pick', 'hat', 'sti', 'ou', 'roc', 'mat', 'st', 'co', 'bro', 'yo']
['s

In [34]:
len(profile_tags)

612

In [35]:
df_profile["tags"] = profile_tags

In [36]:
df_profile.head()

,user_Id,cluster_Id,answer_date,answers,tags
0,54d4a1d6db8155e6700f853b,12,2015-02-06 11:14:00,I'm with family this week and so not super act...,"[git, ant, com, nt, import, https, port, ear, ..."
1,54d4a1d6db8155e6700f853b,61,2015-02-24 22:46:00,cool\nok\n,[co]
2,54d4a1d6db8155e6700f853b,69,2015-02-25 18:59:00,yeah that sounds like a good idea\n,"[ide, go, hat, ou, od, goo, de]"
3,54d4a1d6db8155e6700f853b,72,2015-02-25 19:14:00,you mean == 1\n,"[mean, ou, yo]"
4,54d4a1d6db8155e6700f853b,75,2015-02-25 19:19:00,that is also weird ;)\n,"[ls, rd, hat, lso, ei, als]"


In [37]:
df_profile.to_csv(channel1+'/'+channel1+'_profile_tags.csv',index = None)

In [ ]:
df_issue = tags_match(df_issue["content"],df_tags["TagName"])

[]
['ide', 'nt', 'vi', 'ip', 'int', 'text', 'types', 'integer', 'using', 'date', 'class', 'ice', 'ssi', 'set', 'tex', 'ri', 'tr', 'dvi', 'ace', 'xt', 'ipl', 'som', 'ata', 'ex', 'mu', 'ed', 'ss', 'rs', 'om', 'cl', 'ats', 'lti', 'las', 'know', 'de', 'ta']
['nt', 'binary', 'il', 'date', 'el', 'xa', 'col', 'live', 'word', 'rd', 'ri', 'rt', 'alu', 'bin', 'umn', 'ampl', 'mo', 'ex', 'na', 'ord', 'ril', 'st', 'rs', 'co', 'ch', 'irs', 'ert', 'rise', 'val', 'gr']
['nt', 'message', 'sa', 'sage', 'ssa', 'ess', 'ou', 'der', 'sag', 'ss', 'co', 'ag', 'yo', 'ent', 'de']
['nt', 'vi', 'ip', 'io', 'rest', 'int', 'ear', 'stack', 'trac', 'message', 'action', 'nio', 'ls', 'flow', 'extract', 'el', 'overflow', 'xa', 'avi', 'col', 'sa', 'vin', 'act', 'extraction', 'typed', 'sta', 'quest', 'tr', 'sage', 'lso', 'pan', 'ssa', 'umn', 'ack', 'sti', 'xt', 'ode', 'ess', 'heterogeneous', 'cod', 'rom', 'ampl', 'ets', 'ou', 'ata', 'ex', 'ning', 'ed', 'extra', 'sni', 'pandas', 'st', 'sag', 'ss', 'co', 'om', 'sp', 'ag', '

['git', 'irc', 'rc', 'rom', 'ou', 'om']
['vi', 'invite', 'ag', 'vite']
['string', 'docstring', 'doc', 'ocs', 'cstring', 'su', 'ri', 'tr', 'ring', 'ou', 'gree', 'st', 'ss', 'ag', 'gr']
['relate', 'skin', 'el', 'ref', 'sh', 'ld', 'su', 'bit', 'refer', 'go', 'rel', 'ri', 'rt', 'led', 'hat', 'ode', 'cod', 'ou', 'ed', 'rr', 'ogr', 'st', 'ss', 'co', 'ch', 'jus', 'od', 'kin', 'rise', 'ogl', 'gr', 'goo', 'de']
['ide', 'io', 'ini', 'relate', 'unc', 'el', 'object', 'su', 'go', 'rel', 'hat', 'ning', 'ed', 'plai', 'pbm', 'ss', 'opl', 'ch', 'bunch', 'od', 'goo', 'bun', 'de']
['python', 'string', 'vi', 'https', 'pi', 'native', 'ignore', 'docstring', 'doc', 'http', 'el', 'ati', 'avi', 'ocs', 'nat', 'cstring', 'nose', 'ps', 'ld', 'ri', 'tr', 'pypi', 'tps', 'ring', 'ou', 'na', 'lte', 'pyp', 'tern', 'st', 'co', 'ose', 'gn', 'yt', 'ravi']
['io', 'il', 'config', 'makefile', 'file', 'build', 'pac', 'sa', 'nose', 'ld', 'package', 'sta', 'pack', 'option', 'ack', 'ou', 'ed', 'po', 'efi', 'rep', 'st', 'eve', '

['com', 'nt', 'dep', 'orm', 'ndepend', 'transform', 'rm', 'el', 'em', 'rd', 'let', 'pen', 'tr', 'rom', 'ou', 'tel', 'nsform', 'co', 'om', 'sf', 'ompl', 'ent', 'de']
['com', 'nt', 'ip', 'io', 'double', 'diff', 'orm', 'relate', 'versions', 'nsis', 'transform', 'rm', 'version', 'ems', 'el', 'ssi', 'em', 'min', 'rect', 'pre', 'su', 'rel', 'lts', 'ctl', 'tr', 'scipy', 'cons', 'hat', 'checked', 'ess', 'rom', 'som', 'ou', 'imp', 'sis', 'ed', 'rr', 'scip', 'nsform', 'st', 'eve', 'nco', 'ss', 'rs', 'co', 'ff', 'om', 'sc', 'rro', 'ch', 'sf', 'eme', 'las', 'mi', 'ubl', 'hey', 'ent', 'rsi', 'de']
['ip', 'io', 'il', 'cas', 'row', 'version', 'ca', 'ase', 'case', 'sh', 'su', 'scipy', 'rt', 'ttl', 'cat', 'hat', 'mo', 'ou', 'ning', 'ed', 'rr', 'scip', 'st', 'rs', 'om', 'sc', 'rro', 'yo', 'ch', 'shuttle', 'tom', 'od', 'rsi', 'wil', 'ill']
['io', 'diff', 'orm', 'nic', 'iso', 'regression', 'using', 'transform', 'edi', 'rm', 'ssi', 'etw', 'pre', 'light', 'sli', 'tr', 'hat', 'predict', 'tween', 'ess', 'ed',

['vi', 'row', 'view', 'el', 'viewer', 'review', 'go', 'ri', 'mo', 'ou', 'ed', 'rr', 'om', 'ose', 'rro', 'yo', 'tom', 'od', 'goo']
['vi', 'io', 'class', 'view', 'ca', 'ssi', 'review', 'go', 'uss', 'ready', 'ou', 'lr', 'ss', 'sc', 'cl', 'yo', 'dis', 'got', 'ei', 'las']
['vi', 'ls', 'views', 'view', 'el', 'review', 'som', 'ed', 'om', 'eviews', 'ws', 'eon']
['ram', 'amf', 'rt', 'ou', 'ag', 'yo', 'aro', 'gr']
['ide', 'git', 'com', 'https', 'ear', 'merge', 'il', 'sid', 'get', 'pull', 'doc', 'http', 'github', 'el', 'filter', 'thin', 'ocs', 'ps', 'ri', 'tps', 'ou', 'ed', 'ogr', 'scikit-learn', 'lte', 'mer', 'gf', 'co', 'om', 'sc', 'ag', 'yo', 'rise', 'hub', 'gr', 'de']
[]
['git', 'com', 'https', 'ear', 'nic', 'alpha', 'cal', 'pull', 'http', 'github', 'ca', 'ati', 'ps', 'mean', 'su', 'ri', 'hat', 'tps', 'ring', 'ou', 'ning', 'scikit-learn', 'mer', 'st', 'co', 'om', 'sc', 'cl', 'ag', 'glom', 'hub', 'know']
['testing', 'reduce', 'il', 'move', 'el', 'em', 'thin', 'ld', 'partial', 'rt', 'hat', 'par

['ant', 'nt', 'free', 'port', 'nic', 'iso', 'backport', 'el', 'wan', 'back', 'ri', 'rt', 'ack', 'take', 'ogr', 'po', 'tonic', 'st', 'ff', 'om', 'fee', 'eel', 'ton', 'uff', 'tuf', 'stuff', 'rise', 'soto', 'gr', 'ta']
['io', 'merge', 'nic', 'il', 'iso', 'cal', 'ca', 'probability', 'ati', 'calibration', 'thin', 'abi', 'rect', 'ou', 'ed', 'rr', 'mer', 'tonic', 'st', 'co', 'brat', 'lib', 'babili', 'ton', 'libra', 'sts', 'soto', 'lit']
['merge', 'ou', 'mer', 'rs', 'yo']
['git', 'com', 'https', 'ear', 'pull', 'http', 'github', 'ase', 'base', 'rebase', 'thin', 'ps', 'tps', 'scikit-learn', 'co', 'om', 'sc', 'hub']
['git', 'com', 'beta', 'https', 'port', 'ear', 'warnings', 'backport', 'feedback', 'pull', 'http', 'github', 'war', 'em', 'feed', 'sh', 'ps', 'back', 'ld', 'wait', 'rt', 'hat', 'ack', 'tps', 'ou', 'ning', 'ed', 'po', 'edb', 'scikit-learn', 'co', 'om', 'sc', 'fee', 'eta', 'hub', 'ta']
['vi', 'avi', 'vin', 'go', 'wait', 'ou', 'rr', 'yo']
['python', 'git', 'ant', 'com', 'testing', 'nt', 

...

['nt', 'trace', 'trac', 'ase', 'back', 'ld', 'su', 'pen', 'ri', 'tr', 'ace', 'rt', 'ack', 'ou', 'traceback', 'ss', 'ag', 'yo', 'ent', 'hub', 'eas', 'ta']
['git', 'com', 'https', 'ear', 'cas', 'pi', 'http', 'ping', 'github', 'ca', 'ase', 'case', 'el', 'mpi', 'request', 'ps', 'su', 'go', 'pen', 'quest', 'tps', 'ed', 'bump', 'scikit-learn', 'st', 'ss', 'co', 'om', 'sc', 'ag', 'que', 'got', 'req', 'ump', 'hub', '20']
['nt', 'io', 'get', 'release', 'ca', 'ase', 'core', 'dd', 'el', 'sa', 'su', 'rel', 'rom', 'som', 'att', 'ou', 'ed', 'easing', 'lto', 'red', 'st', 'ss', 'co', 'om', 'yo', 'kin', 'mi', 'ton', 'kind', 'ent', 'orking', 'eas', 'de']
['ning']
['theory', 'nt', 'io', 'ear', 'uri', 'native', 'using', 'suite', 'el', 'ati', 'thin', 'nat', 'rc', 'su', 'alter', 'tor', 'ri', 'tr', 'drag', 'ring', 'ou', 'ed', 'der', 'alt', 'na', 'lte', 'tern', 'st', 'eve', 'ff', 'sc', 'torch', 'ag', 'ch', 'orc', 'jus', 'lib', 'curio', 'fee', 'kin', 'yt', 'rio', 'kind', 'ragg', 'ent', 'sui', 'ory', 'de']
['th

['sed', 'ca', 'metric', 'ri', 'tr', 'rom', 'ed', 'om', 'metr']
['ip', 'io', 'mixed', 'encode', 'rsa', 'lf', 'ati', 'sa', 'bit', 'tr', 'ode', 'cod', 'ou', 'ed', 'rr', 'sni', 'st', 'nco', 'rs', 'co', 'cl', 'yo', 'sat', 'sn', 'gn', 'od', 'xed', 'mi', 'az', 'de']
['el', 'ri', 'ogr', 'rs', 'rise', 'gr']
['com', 'precision', 'io', 'https', 'post', 'stack', 'curve', 'pi', 'get', 'cal', 'class', 'http', 'function', 'ca', 'unc', 'call', 'func', 'pre', 'ps', 'pic', 'sta', 'quest', 'ack', 'sti', 'ode', 'cod', 'tps', 'rom', 'stackexchange', 'ex', 'ed', 'po', 'st', 'ss', 'co', 'om', 'cl', 'ch', 'precision-recall', 'que', 'od', 'tac', 'las', 'pos', 'how-to', 'de', 'ta']
['html', 'io', 'https', 'ear', 'spl', 'label', 'metrics', 'module', 'cal', 'http', 'edi', 'ml', 'ca', 'el', 'metric', 'call', 'sh', 'pre', 'ps', 'ld', 'rate', 'tor', 'sta', 'ri', 'tr', 'prediction', 'predict', 'du', 'sti', 'either', 'tps', 'rom', 'mo', 'ou', 'generated', 'ed', 'po', 'scikit-learn', 'tm', 'mat', 'red', 'st', 'om', 'sc

['ant', 'nt', 'vi', 'ip', 'io', 'environment', 'ram', '64-bit', 'ear', 'post', 'plot', 'il', 'location', 'message', 'action', 'faults', 'default', 'env', 'exe', 'cut', 'return', 'stdout', 'file', 'ca', 'core', 'put', 'amf', 'pts', 'pac', 'std', 'execute', 'messages', 'ati', 'exec', 'matplotlib', 'sh', 'sa', 'wan', 'show', 'pre', 'back', 'stalled', 'act', 'package', 'bit', 'sta', 'cc', 'lts', 'ri', 'rome', 'tr', 'stderr', 'scipy', 'pack', 'fault', 'led', 'cat', 'sage', 'hat', 'iron', 'pan', 'ssa', 'sac', 'ack', 'urn', 'ode', 'ess', 'cod', 'rom', 'defaults', 'ou', 'tde', 'ex', 'ed', 'rr', 'der', 'ogr', 'po', 'nm', 'scip', 'pandas', 'na', 'envi', 'otl', 'mfi', 'ransac', 'scikit-learn', 'nme', 'output', 'mat', 'anaconda', 'red', 'st', 'conda', 'sag', 'ss', 'turn', 'co', 'om', 'sc', 'tput', 'ag', 'rro', 'tal', 'environ', 'lib', 'lli', 'od', 'pos', 'tpu', 'ws', 'anaconda3', 'sly', 'st-link', 'script', 'ent', 'gr', 'de', 'ta']
['com', 'nt', 'sam', 'ase', 'ati', 'em', 'sa', 'eda', 'su', 'sti',

['git', 'io', 'https', 'ear', 'il', 'png', 'ria', 'var', 'image', 'http', 'sed', 'file', 'ca', 'object', 'standardized', 'ps', 'rd', 'sta', 'ri', 'quest', 'variance', 'sti', 'thumb', 'mage', 'bc', 'tps', 'ring', 'pca', 'ou', 'ata', 'ed', 'andar', 'stan', 'st', 'rs', 'ag', 'yo', 'que', 'ether', 'ta', '20']
['git', 'com', 'nt', 'ip', 'io', 'orm', 'session', 'ear', 'plugins', 'il', 'platform', 'counter', 'fork', 'config', 'file', 'rm', 'kill', 'version', 'darwin', 'path', 'unc', 'put', 'el', 'ssi', 'count', 'set', 'tar', 'col', 'root', 'stalled', 'dir', 'go', 'sta', 'rt', 'led', 'tform', 'stp', 'collect', 'ess', 'ou', 'pytest', 'ed', 'ppl', 'scikit-learn', 'owin', 'tup', 'gf', 'red', 'st', 'eve', 'nco', 'mpu', 'ss', 'rs', 'co', 'om', 'sc', 'xdist', 'fig', 'roo', 'ch', 'tal', 'ert', 'plug', 'dis', 'hip', 'got', 'rke', 'yt', 'tdi', 'velo', 'ent', 'rsi', 'cfg', 'ill', 'de', 'ta']
['go', 'od', 'goo']
['python', 'ide', 'git', 'ip', 'io', 'import', 'time', 'port', 'ear', 'hook', 'zip', 'ini', '

['il', 'ca', 'ati', 'rar', 'bit', 'partial', 'ri', 'tr', 'rt', 'hat', 'par', 'arb', 'ril', 'st', 'eas']
['com', 'nt', 'twitter', 'vi', 'io', 'https', 'ear', 'ini', 'il', 'ls', 'ups', 'http', 'ca', 'el', 'status', 'ice', 'init', 'war', 'ati', 'repr', 'ffi', 'em', 'thin', 'rc', 'word', 'die', 'pre', 'ps', 'hour', 'rd', 'tor', 'sta', 'pen', 'ri', 'stat', 'tr', 'tie', 'hat', 'lso', 'ack', 'erl', 'tps', 'rom', 'mo', 'spread', 'ou', 'comm', 'esent', 'mu', 'ed', 'rr', 'der', 'rete', 'mmu', 'ord', 'scikit-learn', 'rep', 'st', 'rs', 'co', 'ff', 'om', 'retention', 'sc', 'opl', 'sp', 'ch', 'munit', 'slack', 'biweekly', 'sente', 'tus', 'als', 'ent', 'wil', 'gr', 'group', 'ill', 'rce', 'de', 'ta']
['sum', 'ati', 'ld', 'su', 'clos', 'hat', 'som', 'mo', 'ou', 'ed', 'ss', 'co', 'om', 'ose', 'cl', 'moti', 'de']
['ide', 'ant', 'com', 'nt', 'algorithm', 'io', 'methods', 'comparison', 'ram', 'ear', 'lua', 'il', 'label', 'metrics', 'iso', 'warnings', 'final', 'sse', 'ls', 'exe', 'sid', 'get', 'using', 'eva

In [ ]:
df_issue